In [1]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#必要なモジュールのインストール
%rm -r /content/ComfyUI-AdvancedLivePortrait
%cd /content/
!git clone https://github.com/PowerHouseMan/ComfyUI-AdvancedLivePortrait.git
%cd /content/ComfyUI-AdvancedLivePortrait
!pip install -r requirements.txt
%cd /content/

#サーバとして利用するためのモジュールのインストール
!pip install -q fastapi\[all\] nest-asyncio uvicorn pyngrok

rm: cannot remove '/content/ComfyUI-AdvancedLivePortrait': No such file or directory
/content
Cloning into 'ComfyUI-AdvancedLivePortrait'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 249 (delta 42), reused 44 (delta 32), pack-reused 184 (from 1)
Receiving objects: 100% (249/249), 14.18 MiB | 36.57 MiB/s, done.
Resolving deltas: 100% (109/109), done.
/content/ComfyUI-AdvancedLivePortrait
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [3]:

from google.colab import userdata
NGROK = userdata.get('NGROK')

# カレントディレクトリを本ファイルが存在するディレクトリに変更する。
import glob
import os
pwd = os.path.dirname(glob.glob('/content/drive/MyDrive/alp_backend/alp_api_backend.ipynb', recursive=True)[0])
print(pwd)

%cd $pwd
!pwd

/content/drive/MyDrive/alp_backend
/content/drive/MyDrive/alp_backend
/content/drive/MyDrive/alp_backend


In [4]:
import sys
sys.path.append("/content/ComfyUI-AdvancedLivePortrait")


from fastapi import FastAPI, UploadFile,File, Form
from fastapi import HTTPException
from fastapi.middleware.cors import CORSMiddleware
from PIL import Image
import io
import os
from fastapi.responses import FileResponse, JSONResponse, StreamingResponse
from module.advanced_live_portrait import AdvancedLivePortrait_execution_prepare, AdvancedLivePortrait_execution_main

In [5]:
app = FastAPI()

os.makedirs("/content/tmp", exist_ok=True)

In [6]:
# CORSの設定
origins = [
    "*" # フロントエンドのURLを指定
    # 必要に応じて他のオリジンを追加
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],  # すべてのHTTPメソッドを許可
    allow_headers=["*"],  # すべてのヘッダーを許可
)

editor = None
img_tensor = None
prepared_face = None


@app.post("/initialize")
async def initialize_image_endpoint(file: UploadFile = File(...)):
    global editor, img_tensor, prepared_face
    # アップロードされた画像を一時ファイルに保存
    input_image_path = f"./tmp/{file.filename}"
    output_image_path = f"./tmp/initialized_{file.filename}"
    os.makedirs(os.path.dirname(input_image_path), exist_ok=True)

    with open(input_image_path, "wb") as f:
        f.write(await file.read())

    # 初期化処理を実行
    editor, img_tensor, prepared_face = AdvancedLivePortrait_execution_prepare(input_image_path, output_image_path)

    # 初期化完了メッセージを返す
    return JSONResponse(content={"message": "初期化が完了しました", "status": "success"})

# 画像編集関数（AdvancedLivePortrait_executionのラッパー）
def edit_image(output_path, parameters):
    # ここにAdvancedLivePortrait_execution関数を呼び出すコードを記載
    # parametersはNext.jsから送られてくるパラメータ
    edited_image_pil = AdvancedLivePortrait_execution_main(editor, img_tensor, prepared_face, output_path, parameters)
    # メモリバッファに画像を保存
    img_byte_arr = io.BytesIO()
    edited_image_pil.save(img_byte_arr, format='PNG')  # PNG形式で保存（JPEGでも可）
    img_byte_arr.seek(0)  # バッファの先頭にポインタを移動

    return img_byte_arr

@app.post("/edit")
async def edit_image_endpoint(
    file: UploadFile = File(...),
    rotate_pitch: float = Form(0),
    rotate_yaw: float = Form(0),
    rotate_roll: float = Form(0),
    blink: float = Form(0),
    eyebrow: float = Form(0),
    wink: float = Form(0),
    pupil_x: float = Form(0),
    pupil_y: float = Form(0),
    aaa: float = Form(0),
    eee: float = Form(0),
    woo: float = Form(0),
    smile: float = Form(0.5),
):
    # アップロードされた画像を一時ファイルに保存
    input_image_path = f"./tmp/{file.filename}"
    output_image_path = f"./tmp/edited_{file.filename}"

    with open(input_image_path, "wb") as f:
        f.write(await file.read())

    # パラメータをAdvancedLivePortrait_executionに渡して編集を行う
    parameters = [
        rotate_pitch, rotate_yaw, rotate_roll, blink, eyebrow, wink,
        pupil_x, pupil_y, aaa, eee, woo, smile
    ]

    print(f"Received parameters: {parameters}")
    img_byte_arr = edit_image(output_image_path, parameters)

    # 編集された画像を返す
    #return FileResponse(output_image_path)
    return StreamingResponse(img_byte_arr, media_type="image/png")

In [7]:
import uvicorn
from pyngrok import ngrok
import nest_asyncio

ngrok.set_auth_token(NGROK)

ngrok_tunnel=ngrok.connect(8000)
print('PUBLIC_URL:',ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app,port=8000)

PUBLIC_URL: https://140b-34-82-25-164.ngrok-free.app


INFO:     Started server process [1038]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Received parameters: [-14.0, -18.0, 15.0, 11.0, 20.0, 1.0, -1.0, -17.0, 0.0, 1.0, 0.0, 0.2]
start running
INFO:     2400:4050:b5a1:9000:b4b1:e15d:6ee7:67ab:0 - "POST /edit HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 401, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 165, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dis

(512, 512)
Uses CUDA device.
torch.Size([1, 512, 512, 3])
load image
load LP_Engine
Prepare source...
Load pipeline...
AdvancedLivePortrait: Downloading model...


...

Downloading: 100%|██████████| 890k/890k [00:00<00:00, 16.8MiB/s]


AdvancedLivePortrait: Downloading model...


Downloading: 100%|██████████| 5.94M/5.94M [00:00<00:00, 69.5MiB/s]


Image dimensions before YOLO: torch.Size([1, 3, 512, 512])

0: 512x512 1 face, 93.3ms
Speed: 1.0ms preprocess, 93.3ms inference, 997.4ms postprocess per image at shape (1, 3, 512, 512)
w, h:(512, 512)
diff, min_diff, best_box:(35.19403076171875, 512, [154.57788, 108.6785, 287.03406, 276.0973])
load ExpressionEditor
INFO:     2400:4050:b5a1:9000:b4b1:e15d:6ee7:67ab:0 - "POST /initialize HTTP/1.1" 200 OK
Received parameters: [-14.0, -18.0, 15.0, 11.0, 20.0, 1.0, -1.0, -17.0, 0.0, 1.0, 0.0, 0.2]
start running
Load pipeline...
メイン処理実行時間： 2.2141079902648926
finish running
INFO:     2400:4050:b5a1:9000:b4b1:e15d:6ee7:67ab:0 - "POST /edit HTTP/1.1" 200 OK
(512, 512)
torch.Size([1, 512, 512, 3])
load image
load LP_Engine
Prepare source...
Load pipeline...
Image dimensions before YOLO: torch.Size([1, 3, 512, 512])

0: 512x512 1 face, 6.7ms
Speed: 0.9ms preprocess, 6.7ms inference, 2.8ms postprocess per image at shape (1, 3, 512, 512)
w, h:(512, 512)
diff, min_diff, best_box:(35.19403076171875, 

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1038]


1 girl ,Yellowish-white hair ,short hair ,red small ribbon,red eyes,red hat ,school uniform ,solo ,smile ,upper body ,Anime ,Japanese,best quality,high quality,ultra highres,ultra quality


nsfw, lowres, (bad), text, error, fewer, extra, missing, worst quality, jpeg artifacts, low quality, watermark, unfinished, displeasing, oldest, early, chromatic aberration, signature, extra digits, artistic error, username, scan, [abstract]